In [106]:
import pandas as pd
import pymongo
import json
from pandas import DataFrame

In [84]:
client = pymongo.MongoClient("mongo:27017")
database = client['TripAdvisor']
collection = database['TA_Restaurant']

In [85]:
df = pd.read_csv('TA_restaurants_curated.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df = df.dropna()
df = df.drop_duplicates(subset='ID_TA', keep="first")

### Importer les données

In [86]:
df= df.rename(columns={"ID_TA":"_id"})

In [87]:
payload = json.loads(df.to_json(orient='records'))
collection.delete_many({}) #pour nettoyer la collection
collection.insert_many(payload)

## Requêtes 

In [88]:
# nombre de villes différentes
len(collection.distinct('City'))

31

In [89]:
# Les 10 meilleurs restaurants
top10 = collection.find().sort([("Ranking",1),("Rating",-1),("Number of Reviews",-1)]).limit(10)
list(top10)

[{'_id': 'd794237',
  'Name': 'Comme Chez Soi',
  'City': 'Budapest',
  'Cuisine Style': "['Italian', 'Mediterranean', 'European', 'Hungarian', 'Vegetarian Friendly', 'Gluten Free Options']",
  'Ranking': 1.0,
  'Rating': 5.0,
  'Price Range': '$$ - $$$',
  'Number of Reviews': 4437.0,
  'Reviews': "[['Great place!', 'Best best best'], ['01/07/2018', '01/05/2018']]",
  'URL_TA': '/Restaurant_Review-g274887-d794237-Reviews-Comme_Chez_Soi-Budapest_Central_Hungary.html'},
 {'_id': 'd8374547',
  'Name': "Wine O'Clock Shop Prague",
  'City': 'Prague',
  'Cuisine Style': "['Italian', 'Mediterranean', 'European', 'Wine Bar', 'International', 'Vegetarian Friendly', 'Gluten Free Options']",
  'Ranking': 1.0,
  'Rating': 5.0,
  'Price Range': '$$ - $$$',
  'Number of Reviews': 967.0,
  'Reviews': "[['Best dinner in prague!', 'Lovely evening'], ['01/09/2018', '01/07/2018']]",
  'URL_TA': '/Restaurant_Review-g274707-d8374547-Reviews-Wine_O_Clock_Shop_Prague-Prague_Bohemia.html'},
 {'_id': 'd800303

In [116]:
nb_price = collection.aggregate([
    {"$group" : {
        "_id" : "$City",
        "count_price" : {"$sum": 1},
    }}
])
list_cur= list(nb_price)
list_cur

[{'_id': 'Dublin', 'count_price': 1460},
 {'_id': 'Helsinki', 'count_price': 754},
 {'_id': 'Lyon', 'count_price': 1376},
 {'_id': 'Budapest', 'count_price': 1640},
 {'_id': 'Berlin', 'count_price': 3438},
 {'_id': 'Milan', 'count_price': 3737},
 {'_id': 'Oporto', 'count_price': 1013},
 {'_id': 'Ljubljana', 'count_price': 317},
 {'_id': 'Munich', 'count_price': 1679},
 {'_id': 'Warsaw', 'count_price': 1232},
 {'_id': 'Copenhagen', 'count_price': 1273},
 {'_id': 'Stockholm', 'count_price': 1364},
 {'_id': 'Paris', 'count_price': 9281},
 {'_id': 'Prague', 'count_price': 2527},
 {'_id': 'Madrid', 'count_price': 4599},
 {'_id': 'Geneva', 'count_price': 954},
 {'_id': 'Hamburg', 'count_price': 1197},
 {'_id': 'Oslo', 'count_price': 749},
 {'_id': 'Athens', 'count_price': 1166},
 {'_id': 'Luxembourg', 'count_price': 437},
 {'_id': 'London', 'count_price': 11622},
 {'_id': 'Rome', 'count_price': 4654},
 {'_id': 'Edinburgh', 'count_price': 1404},
 {'_id': 'Amsterdam', 'count_price': 2426},
 {'

In [109]:
df = DataFrame(list_cur)

In [110]:
test_csv_data = df.to_csv('test.csv', index = True)